## Dataset prep for Llama-3.1

alpaca format

```
{"instruction": "...", "input": "...", "output": "..."}
```



In [8]:
import json
import tiktoken
import numpy as np
from datasets import Dataset, load_dataset
import pandas as pd

In [31]:
pd.set_option("display.max_colwidth", 1000)

In [28]:
# dataset_path = "../../dataset/nl-logql-dataset-03"
dataset = load_dataset("sidbin/natural-logql")

df = dataset["train"].to_pandas()

In [29]:
train_df = dataset["test"].to_pandas()

In [32]:
train_df

,id,application,application_variables,category,label_filter,line_filter,log_category_result,logql_query,metric_category,metric_category_result,query_explanation,query_result,question,row_variables,variables
0,0,openstack,[application],,multiple log stream selectors,multiple line filters,"{'chain_of_thought': 'Analyzing the query, it contains three label filters: `application=""openstack""`, `log_file_type=""nova-compute""`, `component=""nova.compute.manager""`. Additionally, there are three line filters (`|= ""3edec1e4-9678-4a3a-a21b-a145a4ee5e61""`, `|= ""Took""`, `|= ""seconds to spawn the instance on the hypervisor""`) followed by a regex operation and a line format operation. The presence of three label filters classifies this under multiple log stream selectors, and the presence of more than one line filter classifies it under multiple line filters.', 'label_filter': 'multiple log stream selectors', 'line_filter': 'multiple line filters'}","{application=""openstack-us-east"", log_file_type=""nova-compute"", component=""nova.compute.manager""} |= ""3edec1e4-9678-4a3a-a21b-a145a4ee5e61"" |= ""Took"" |= ""seconds to spawn the instance on the hypervisor"" | regexp ""\\[instance: (?P<instance_id>[^\\]]+)\\] Took (?P<spawn_time>\\d+\\.\\d+) seconds to spawn the instance on the hypervisor"" | line_format ""{{.instance_id}} took {{.spawn_time}}""",None,"{'categories': None, 'chain_of_thought': 'The provided query involves log matching and extraction using filters and regexp but does not perform any quantitative aggregations or statistical computations, such as sum, count, or rate on the extracted data. It includes log filters, a regexp pattern for extracting data, and formatting the output using line_format, which suggests manipulation of text rather than numerical computation for metrics. Therefore, there are no metric aggregations present in this query.'}","1\n{application=""openstack"", log_file_type=""nova-compute"", component=""nova.compute.manager""}\nFetch all log lines matching label filters.\n2\n<expr> |= `3edec1e4-9678-4a3a-a21b-a145a4ee5e61`\nReturn log lines that contain string 3edec1e4-9678-4a3a-a21b-a145a4ee5e61.\n\n3\n<expr> |= `Took`\nReturn log lines that contain string Took.\n\n4\n<expr> |= `seconds to spawn the instance on the hypervisor`\nReturn log lines that contain string seconds to spawn the instance on the hypervisor.\n\n5\n<expr> | regexp `\[instance: (?P<instance_id>[^\]]+)\] Took (?P<spawn_time>\d+\.\d+) seconds to spawn the instance on the hypervisor`\nThe regexp parser takes a single parameter | regexp ""<re>"" which is the regular expression using the Golang RE2 syntax. The regular expression must contain a least one named sub-match (e.g (?P<name>re)), each sub-match will extract a different label. The expression matches the structure of a log line. The extracted labels can be used in label filter expressions and ...",3edec1e4-9678-4a3a-a21b-a145a4ee5e61 took 20.58,How long did it take to spawn instance 3edec1e4-9678-4a3a-a21b-a145a4ee5e61 on the hypervisor for openstack-us-east?,"[instance_id, spawn_time]","[instance_id, time_in_sec]"
1,72,openstack,[application],Instance Lifecycle,multiple log stream selectors,multiple line filters,"{'chain_of_thought': 'This query uses three label filters: `application=""openstack""`, `log_file_type=""nova-compute""`, and `component=""nova.compute.manager""`. Additionally, it contains two line filters: `|= ""cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us""` and `|= ""Terminating instance""`. Therefore, the classification includes multiple label filters and multiple line filters.', 'label_filter': 'multiple log stream selectors', 'line_filter': 'multiple line filters'}","sum(count_over_time({application=""openstack-asia-pacific"", log_file_type=""nova-compute"", component=""nova.compute.manager""}\n|= ""cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us""\n|= ""Terminating instance""\n[1d] offset 1d))","[built_in_range_aggregation, log_range_aggregation]","{'categories': ['built_

In [25]:
from typing import Dict, List


def prep_for_llama(row: Dict):
    instruction = "You are LogQLLM, a specialized language model trained to convert natural language queries into LogQL (Log Query Language) queries. Your primary function is to interpret user requests expressed in plain English and translate them into valid LogQL syntax."
    input = row["question"]
    output = f"[QL] {row['logql_query']} [/QL] [EXPLANATION] {row['query_explanation']} [/EXPLANATION]"
    messages = {"instruction": instruction, "input": input, "output": output}
    return messages
    # return {"alpaca": messages}

In [26]:
import os

train_data = dataset["train"].map(
    prep_for_llama,
    num_proc=os.cpu_count(),
    remove_columns=dataset["train"].column_names,
)

Map (num_proc=20):   0%|          | 0/339 [00:00<?, ? examples/s]

In [27]:
train_data.select_columns(["instruction", "input", "output"]).to_json("data.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

569714